# VacationPy

## Setup

In [1]:
import geoviews as gv
import pandas as pd
import requests

import hvplot.pandas # for geoviews

/Users/tyler/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Establish the API key

In [2]:
from api_keys import geoapify_api_key

### Establish the DataFrame

In [3]:
cities_df = pd.read_csv("data/cities.csv")

cities_df.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,271.06,40,0,7.20,SJ,1739728736
1,1,laguna,38.4210,-121.4238,285.08,84,100,1.54,US,1739728737
2,2,kavieng,-2.5744,150.7967,301.23,77,96,3.68,PG,1739728740
3,3,afif,23.9065,42.9172,291.98,55,58,2.97,SA,1739728742
4,4,nova sintra,14.8667,-24.7167,294.89,73,48,6.49,CV,1739728743


## Create maps

### All data

In [4]:
points = gv.Points(cities_df, ["Lon", "Lat"])
points.opts(
    title="Cities (all data)",
    width=1000,
    height=700,
    color="City",
    cmap="Category20",
    size=1+(gv.dim("Humidity")*.2),
    toolbar=None
) * gv.feature.coastline

:Overlay
   .Points.I    :Points   [Lon,Lat]   (City_ID,City,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date)
   .Coastline.I :Feature   [Longitude,Latitude]

### Custom data

#### Create a new DataFrame (filtered data, different columns)

In [5]:
# create a copy
cities_df_custom = cities_df.copy()

cities_df_custom.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,271.06,40,0,7.20,SJ,1739728736
1,1,laguna,38.4210,-121.4238,285.08,84,100,1.54,US,1739728737
2,2,kavieng,-2.5744,150.7967,301.23,77,96,3.68,PG,1739728740
3,3,afif,23.9065,42.9172,291.98,55,58,2.97,SA,1739728742
4,4,nova sintra,14.8667,-24.7167,294.89,73,48,6.49,CV,1739728743


In [6]:
# filter the data
cities_df_custom = cities_df_custom[(cities_df_custom["Humidity"] <= 25) & (cities_df_custom["Cloudiness"] <= 50) & (cities_df_custom["Wind Speed"] <= 10)]

cities_df_custom.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
70,70,kura,11.7697,8.4303,299.94,11,37,3.99,NG,1739728829
110,110,ati,13.2154,18.3353,300.11,10,32,4.73,TD,1739728879
140,140,yagoua,10.3411,15.2329,303.28,10,48,4.51,CM,1739728920
144,144,chos malal,-37.3781,-70.2709,302.89,7,0,7.75,AR,1739728925
149,149,batie,9.8833,-2.9167,307.95,11,25,1.90,BF,1739728931


In [7]:
# change the columns
cities_df_custom = cities_df_custom[["City", "Country", "Lat", "Lon", "Humidity"]]
cities_df_custom["Hotel Name"] = None

cities_df_custom

,City,Country,Lat,Lon,Humidity,Hotel Name
70,kura,NG,11.7697,8.4303,11,None
110,ati,TD,13.2154,18.3353,10,None
140,yagoua,CM,10.3411,15.2329,10,None
144,chos malal,AR,-37.3781,-70.2709,7,None
149,batie,BF,9.8833,-2.9167,11,None
156,taoudenni,ML,22.6783,-3.9836,8,None
169,eduardo castex,AR,-35.9150,-64.2945,8,None
208,nguigmi,NE,14.2495,13.1092,11,None
301,bitkine,TD,11.9801,18.2138,11,None
319,mora,CM,11.0461,14.1401,13,None


#### Get the data for the new column "Hotel Name"

In [8]:
API_url = "https://api.geoapify.com/v2/places"

radius = 10000
params = {
    "filter": None,
    "bias": None
}

for index, row in cities_df_custom.iterrows():

    city_name = row["City"]

    # get lat and lon
    city_lat = row["Lat"]
    city_lon = row["Lon"]

    # create parameters
    params["filter"] = f"circle:{city_lon},{city_lat},{radius}"
    params["bias"] = f"proximity:{city_lon},{city_lat}"

    # construct the url
    city_url = f"{API_url}?categories=accommodation.hotel&filter={params['filter']}&bias={params['bias']}&apiKey={geoapify_api_key}"

    try:

        response = requests.get(city_url)
        data = response.json()

        hotel_name = data["features"][0]["properties"]["name"]
        cities_df_custom.loc[index, "Hotel Name"] = hotel_name
        print(f"closest hotel: {hotel_name}")

    except Exception as e:
        cities_df_custom.loc[index, "Hotel Name"] = "Hotel not found"
        print(f"{e} (skipping...)")
        pass

cities_df_custom

list index out of range (skipping...)
list index out of range (skipping...)
closest hotel: Mervane hôtel
closest hotel: Hospedaje Lemus
list index out of range (skipping...)
list index out of range (skipping...)
closest hotel: Hotel San Cristóbal
closest hotel: Guest PAM
list index out of range (skipping...)
closest hotel: Hôtel Podoko
closest hotel: Gesar Hotel
closest hotel: Nora Hôtel
closest hotel: Hôtel Lafi 2*
list index out of range (skipping...)
closest hotel: Hotel Mazamet
closest hotel: Sanaag Hotel


,City,Country,Lat,Lon,Humidity,Hotel Name
70,kura,NG,11.7697,8.4303,11,Hotel not found
110,ati,TD,13.2154,18.3353,10,Hotel not found
140,yagoua,CM,10.3411,15.2329,10,Mervane hôtel
144,chos malal,AR,-37.3781,-70.2709,7,Hospedaje Lemus
149,batie,BF,9.8833,-2.9167,11,Hotel not found
156,taoudenni,ML,22.6783,-3.9836,8,Hotel not found
169,eduardo castex,AR,-35.9150,-64.2945,8,Hotel San Cristóbal
208,nguigmi,NE,14.2495,13.1092,11,Guest PAM
301,bitkine,TD,11.9801,18.2138,11,Hotel not found
319,mora,CM,11.0461,14.1401,13,Hôtel Podoko


#### Plot the data with a hover message

In [9]:
points = gv.Points(cities_df_custom, ["Lon", "Lat"])
points.opts(
    title="Cities (all data)",
    width=1000,
    height=700,
    color="City",
    cmap="Category20",
    size=1+(gv.dim("Humidity")*.2),
    tools=["hover"],
    toolbar=None
) * gv.feature.coastline

:Overlay
   .Points.I    :Points   [Lon,Lat]   (City,Country,Humidity,Hotel Name)
   .Coastline.I :Feature   [Longitude,Latitude]